# pysam
Pysam is a python module that makes it easy to read and manipulate mapped short read sequence data stored in SAM/BAM files. It is a lightweight wrapper of the htslib C-API.

This page provides a quick introduction in using pysam followed by the API. See Working with BAM/CRAM/SAM-formatted files for more detailed usage instructions.
## Installation
I installed Bioconda (https://bioconda.github.io/) trough anaconda.
### Bioconda
Bioconda is a distribution of bioinformatics software realized as a channel for the versatile Conda package manager. Key features of Conda are
* a command line client for simple installation and dependency handling in the spirit of conda install mypackage,
very easy package creation,
* a mechanism for creating isolated environments that allows different package versions to coexist.

#### Step 1 Setup Bioconda
After installing Miniconda, you can use the conda command to setup bioconda with:

The r channel is added to satisfy the R language dependencies of some packages as well as some non-R dependencies. Even if you don’t plan on installing R packages, the r channel is required for some Bioconda packages.
#### Step 2 Install pysam
Once the channels have been added, you can install packages, e.g.:

In [ ]:
conda install pysam

## Use
The examples are based on http://pysam.readthedocs.io/en/latest/api.html. The api are described here:http://pysam.readthedocs.io/en/latest/api.html#api
### Read a bam file
Read a file in BAM format and create an AlignmentFile object.

In [36]:
import pysam

bamfile38 = pysam.AlignmentFile("hg38.bam", "rb")
#bamfile19 = pysam.AlignmentFile("hg19.bam", "rb")

If you want to read a sam file simply use the  'r' without the binary flag.
### Iterate over the alignment
Once a file is opened you can iterate over all of the read mapping to a specified region using fetch(). It works on indexed bam

In [65]:
count=1
for read in bamfile38.fetch('chr19', 46381104, 46381105):
    print read.query_name,read.cigarstring,\
    str(read.is_read1),str(read.is_proper_pair),\
    str(read.is_secondary),str(read.flag),\
    str(read.cigartuples)
    count=count+1
    if count>10:
        break


#samfile38.close()

M00303:129:000000000-AMRLV:1:2110:9807:7376 250M False False False 161 [(0, 250)]
M00303:129:000000000-AMRLV:1:1112:13319:13883 250M False False False 161 [(0, 250)]
M00303:129:000000000-AMRLV:1:1113:14055:28455 241M10S False False False 161 [(0, 241), (4, 10)]
M00303:129:000000000-AMRLV:1:2112:17761:28789 240M11S False True False 163 [(0, 240), (4, 11)]
M00303:129:000000000-AMRLV:1:1107:8661:26872 213M38S True False False 73 [(0, 213), (4, 38)]
M00303:129:000000000-AMRLV:1:1104:7888:13610 201M50S False False False 161 [(0, 201), (4, 50)]
M00303:129:000000000-AMRLV:1:1108:26709:21195 201M50S True True False 99 [(0, 201), (4, 50)]
M00303:129:000000000-AMRLV:1:2112:23718:6595 200M51S False True False 163 [(0, 200), (4, 51)]
M00303:129:000000000-AMRLV:1:2113:17731:20386 199M52S False False False 161 [(0, 199), (4, 52)]
M00303:129:000000000-AMRLV:1:2103:24812:21675 199M52S True True False 99 [(0, 199), (4, 52)]


The read belong to a class pysam.AlignedSegment. Many methods access to the reads features. Here as example we access to: 
* the name of the read
* the cigar string
* the forward/reverse property
* the proper alignment
* the alignment type (secondary/not secondary)
* the flag
* the cigar tuple (M=0,...,S=4,...)

### pysam with mate
Now we search in the bam file the mate reads

In [71]:
def get_SA_items(read):
    SA = [x for x in read.get_tags() if x[0] == 'XA']
    #SA=[x for x in read.get_tags()]
    SA = SA[0] if SA else None
    if not SA:
        return None, None, None, None
    items = SA[1].split(",")
    chrom = items[0]
    pos = int(items[1]) - 1
    #strand = items[2]
    cigar = items[2]
    #total_bases = total_bases_cigar(cigar)
    #sa_mapq = items[4].split(";")[0]
    nm = items[3].split(";")[0]
    return chrom, pos, cigar,nm

CIGAR_SOFT_CLIP = 4
#for read in bamfile38.fetch('chr19', 46381104, 46381105):
for read in bamfile38.fetch('HIV-HXB2'):
    if(read.is_proper_pair):
    #if read.is_secondary:
        #SA_chrom, SA_pos, SA_cigar, SA_nm = get_SA_items(read)
        #print SA_chrom,SA_pos, SA_nm, SA_cigar
        chrom=read.reference_name
        start=read.reference_start+1
        stop=read.reference_end+1
        cigar=read.cigarstring
        mate=bamfile38.mate(read)
        
        chromM=mate.reference_name
        startM=mate.reference_start+1
        stopM=mate.reference_end+1
        cigarM=mate.cigarstring
        print chrom,start,stop,cigar,str(read.is_read1),str(read.is_reverse),\
              chromM,startM,stopM,cigarM,str(mate.is_read1),str(mate.is_reverse)
        #print type(mate)
        #print str(mate)+" "+read.cigarstring+" "+mate.cigarstring